# Project Group 13

Members: Arbman, Kelvin, 4943589; Houterman, Simon; Koetsier, Lars; Linders, Joris



# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

Research Question:

### How was the road safety in Germany affected during the COVID pandemic compared to the previous five years?

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

- Road safety-data in Germany 
    - Incidents resulting in human injury
- COVID-data in Germany
    - Positive tests
    - People on Intensive care (IC)
    - COVID-related deaths 
    
- Timespan: five years before pandemic - 2021 (mid-pandemic)

In [40]:
import pandas as pd
import numpy as np
import math
import scipy
from scipy.stats.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio

import datetime

# Data Pipeline

In [150]:
# Pipeline road Safety Data

# File path
file_path = 'data/road safety germany.csv'

# Open File
df_road_safety_germany = pd.read_csv(file_path,  delimiter=';')

# Adjust Data

# Step 1: This file has the columns Year (YYYY) & Month (mmm)(de), for convience we will add a date column as the table index. 
#         This column will contain the date of the first day of the corresponding month.
# - Step 1.1: Convert the month values (mmm)(de) (e.i. Januar, Februar, März, ..) to month (mm) values (i.e. 01, 02, 03, ..)
df_road_safety_germany = df_road_safety_germany.replace(['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 
                                                         'August', 'September', 'Oktober', 'November', 'Dezember'], 
                                                        ['01','02','03','04','05','06','07','08','09','10','11','12'])

# - Step 1.2: Add [Date] column to dataframe
df_road_safety_germany['Date'] = '01-' + df_road_safety_germany['Month'] + '-' + df_road_safety_germany['Year'].astype(str)
# - Convert string to date value
df_road_safety_germany['Date'] = pd.to_datetime(df_road_safety_germany['Date'], format = '%d-%m-%Y')
# - Set date column as index 
df_road_safety_germany.set_index('Date', inplace=True)

# Step 2: Clean Data, this dataframe contains '...' values for future dates since there is no data available yet.
#         These values will be replaced by empty values 
df_road_safety_germany = df_road_safety_germany.replace({'...': None})



# Step 3: Select only the necesarry columns from the dataframe - We're only interested in the data relating to human injury
df_road_safety_germany = df_road_safety_germany[['Unfälle mit Personenschaden - Insgesamt', 
                                                'Schwerwiegende Unfälle mit Sachschaden i.e.S - Insgesamt',
                                                'Sonst. Unfälle unter dem Einfluss berausch. Mittel - Insgesamt',
                                                'Übrige Sachschadensunfälle - Insgesamt', 
                                                'Insgesamt - Insgesamt']]

# Step 4: Rename column names from German to English
df_road_safety_germany = df_road_safety_germany.rename(columns={
    'Unfälle mit Personenschaden - Insgesamt': 'Accidents involving human injury', 
    'Schwerwiegende Unfälle mit Sachschaden i.e.S - Insgesamt': 'Serious accidents with material damange',
    'Sonst. Unfälle unter dem Einfluss berausch. Mittel - Insgesamt': 'Accidents under the influence of toxins', 
    'Übrige Sachschadensunfälle - Insgesamt': 'Other accidents',
    'Insgesamt - Insgesamt': 'Total accidents'})

# Step 5: Convert to int
#         First drop future dates without values
df_road_safety_germany = df_road_safety_germany.dropna()
df_road_safety_germany = df_road_safety_germany.astype({'Accidents involving human injury': 'int64'})

# Read File
df_road_safety_germany

Accidents involving human injury  \
Date                                           
2011-01-01                             16448   
2011-02-01                             16227   
2011-03-01                             21569   
2011-04-01                             26411   
2011-05-01                             30831   
...                                      ...   
2022-01-01                             16819   
2022-02-01                             16067   
2022-03-01                             21398   
2022-04-01                             20758   
2022-05-01                             27159   

           Serious accidents with material damange  \
Date                                                 
2011-01-01                                    7045   
2011-02-01                                    6138   
2011-03-01                                    5919   
2011-04-01                                    5717   
2011-05-01                                    6099   
...                                            ...   
2022-01-01                                    5731   
2022-02-01                                    4889   
2022-03-01                                    4457   
2022-04-01                                    5082   
2022-05-01                                    4397   

           Accidents under the influence of toxins Other accidents  \
Date                                                                 
2011-01-01                                    1142          151332   
2011-02-01                                    1043          137738   
2011-03-01                                    1114          154508   
2011-04-01                                    1245          156631   
2011-05-01                                    1291          170684   
...                                            ...             ...   
2022-01-01                                    1118          152944   
2022-02-01                                    1110          148594   
2022-03-01                                    1184          164299   
2022-04-01                                    1206          168157   
2022-05-01                                    1139          181329   

           Total accidents  
Date                        
2011-01-01          175967  
2011-02-01          161146  
2011-03-01          183110  
2011-04-01          190004  
2011-05-01          208905  
...                    ...  
2022-01-01          176612  
2022-02-01          170660  
2022-03-01          191338  
2022-04-01          195203  
2022-05-01          214024  

[137 rows x 5 columns]

In [3]:
# Pipeline road Safety Data

# File path
file_path = 'data/road safety germany.csv'
# Open File
df_accidents_human_injury = pd.read_csv(file_path,  delimiter=';')

# Adjust Data

# Step 1: This file has the columns Year (YYYY) & Month (mmm)(de), for convience we will add a date column as the table index. 
#         This column will contain the date of the first day of the corresponding month.
# - Step 1.1: Convert the month values (mmm)(de) (e.i. Januar, Februar, März, ..) to month (mm) values (i.e. 01, 02, 03, ..)
df_accidents_human_injury = df_accidents_human_injury.replace(['Januar', 'Februar', 'März', 'April', 'Mai', 'Juni', 'Juli', 
                                                         'August', 'September', 'Oktober', 'November', 'Dezember'], 
                                                        ['01','02','03','04','05','06','07','08','09','10','11','12'])

# - Step 1.2: Add [Date] column to dataframe
df_accidents_human_injury['Date'] = '01-' + df_accidents_human_injury['Month'] + '-' + df_accidents_human_injury['Year'].astype(str)
# - Convert string to date value
df_accidents_human_injury['Date'] = pd.to_datetime(df_accidents_human_injury['Date'], format = '%d-%m-%Y')
# - Set date column as index 
df_accidents_human_injury.set_index('Date', inplace=True)

# Step 2: Clean Data, this dataframe contains '...' values for future dates since there is no data available yet.
#         These values will be replaced by empty values 
df_accidents_human_injury = df_accidents_human_injury.replace({'...': None})



# Step 3: Select only the necesarry columns from the dataframe - We're only interested in the data relating to human injury
df_accidents_human_injury = df_accidents_human_injury[['Unfälle mit Personenschaden - innerorts', 
                                                       'Unfälle mit Personenschaden - außerorts (ohne Autobahnen)',
                                                       'Unfälle mit Personenschaden - auf Autobahnen',
                                                       'Unfälle mit Personenschaden - Insgesamt', 
                                                      ]]

# Step 4: Rename column names from German to English. 
df_accidents_human_injury = df_accidents_human_injury.rename(columns={
    'Unfälle mit Personenschaden - Insgesamt': 'Accidents involving human injury - total', 
    'Unfälle mit Personenschaden - innerorts': 'Accidents involving human injury - within city limits',
    'Unfälle mit Personenschaden - auf Autobahnen': 'Accidents involving human injury - highway', 
    'Unfälle mit Personenschaden - außerorts (ohne Autobahnen)': 'Accidents involving human injury - outside city limits, off highway',
    })


# Step 5: Convert to int
#         First drop future dates without values
df_accidents_human_injury = df_accidents_human_injury.dropna()
df_accidents_human_injury = df_accidents_human_injury.astype({'Accidents involving human injury - total': 'int64'})
df_accidents_human_injury = df_accidents_human_injury.astype({'Accidents involving human injury - within city limits': 'int64'})
df_accidents_human_injury = df_accidents_human_injury.astype({'Accidents involving human injury - highway': 'int64'})
df_accidents_human_injury = df_accidents_human_injury.astype({'Accidents involving human injury - outside city limits, off highway': 'int64'})

# Read File
df_accidents_human_injury


Accidents involving human injury - within city limits  \
Date                                                                
2011-01-01                                              10351       
2011-02-01                                              10791       
2011-03-01                                              15055       
2011-04-01                                              18390       
2011-05-01                                              22273       
...                                                       ...       
2022-01-01                                              11317       
2022-02-01                                              10884       
2022-03-01                                              15507       
2022-04-01                                              14280       
2022-05-01                                              19568       

            Accidents involving human injury - outside city limits, off highway  \
Date                                                                              
2011-01-01                                               4874                     
2011-02-01                                               4296                     
2011-03-01                                               5236                     
2011-04-01                                               6607                     
2011-05-01                                               7017                     
...                                                       ...                     
2022-01-01                                               4341                     
2022-02-01                                               4011                     
2022-03-01                                               4776                     
2022-04-01                                               5124                     
2022-05-01                                               6206                     

            Accidents involving human injury - highway  \
Date                                                     
2011-01-01                                        1223   
2011-02-01                                        1140   
2011-03-01                                        1278   
2011-04-01                                        1414   
2011-05-01                                        1541   
...                                                ...   
2022-01-01                                        1161   
2022-02-01                                        1172   
2022-03-01                                        1115   
2022-04-01                                        1354   
2022-05-01                                        1385   

            Accidents involving human injury - total  
Date                                                  
2011-01-01                                     16448  
2011-02-01                                     16227  
2011-03-01                                     21569  
2011-04-01                                     26411  
2011-05-01                                     30831  
...                                              ...  
2022-01-01                                     16819  
2022-02-01                                     16067  
2022-03-01                                     21398  
2022-04-01                                     20758  
2022-05-01                                     27159  

[137 rows x 4 columns]

In [4]:
# Pipeline Covid Positive Test

# File path
file_path = 'data/positive covid tests germany.csv'

# Open File
df_positive_covid_test_germany = pd.read_csv(file_path,  delimiter=';')

# Adjust Data
# Step 1: This file has the column Year_Week (YYYY_ww) for convience we will add a date column as the table index. 
#         This column will contain the date of the first day of the corresponding week.
# - Step 1.1: create date column
df_positive_covid_test_germany['Date'] = None
# - Step 1.1: Convert the year and week data to dates using the strptime function. 
#             For the year 2020 the week needs to be offset by 7 days, this is due to different interpretations of a 53th 
#             week in 2019.
for i in range(len(df_positive_covid_test_germany)):
    d = df_positive_covid_test_germany['Year_Week'][i]
    if d.startswith('2020'):
        df_positive_covid_test_germany['Date'][i] = datetime.datetime.strptime(d + '-1', "%Y_%W-%w") - datetime.timedelta(days=7)
    elif d.startswith('2021'):
        df_positive_covid_test_germany['Date'][i] = datetime.datetime.strptime(d + '-1', "%Y_%W-%w")
    else:
        df_positive_covid_test_germany['Date'][i] = datetime.datetime.strptime(d + '-1', "%Y_%W-%w")

# - Set date column as index 
df_positive_covid_test_germany.set_index('Date', inplace=True)

# Step 2: Clean Data



# Step 3: Select only the necesarry columns from the dataframe 
df_positive_covid_test_germany = df_positive_covid_test_germany[['Gesamt']]

# Step 4: Rename column names from German to English
df_positive_covid_test_germany = df_positive_covid_test_germany.rename(columns={
    'Gesamt': 'Number of Positive COVID-19 tests per 100.000 capita'})

# Step 5: Convert the decimal comma to a decimal point, then convert the string to a float, 
df_positive_covid_test_germany["Number of Positive COVID-19 tests per 100.000 capita"]=df_positive_covid_test_germany["Number of Positive COVID-19 tests per 100.000 capita"].str.replace(',','.')
df_positive_covid_test_germany['Number of Positive COVID-19 tests per 100.000 capita'] = df_positive_covid_test_germany['Number of Positive COVID-19 tests per 100.000 capita'].astype(float)

# Read File
df_positive_covid_test_germany

Number of Positive COVID-19 tests per 100.000 capita
Date                                                            
2020-03-02                                               1.09   
2020-03-09                                               7.73   
2020-03-16                                              26.91   
2020-03-23                                              40.81   
2020-03-30                                              43.31   
...                                                       ...   
2022-08-29                                             241.96   
2022-09-05                                             244.93   
2022-09-12                                             274.68   
2022-09-19                                             343.30   
2022-09-26                                             529.81   

[135 rows x 1 columns]

In [5]:
# Pipeline Covid Deaths

# File path
file_path = 'data/covid deaths germany.csv'

# Open File
df_covid_deaths_germany = pd.read_csv(file_path,  delimiter=';')

# Adjust Data
# Step 1: This file has the column Year_Month (YYYY-mm), for convience we will add a date column as the table index. 
#         This column will contain the date of the first day of the corresponding month.
df_covid_deaths_germany['Date'] = df_covid_deaths_germany['Year-Month'] + '-01'
# - Convert string to date value
df_covid_deaths_germany['Date'] = pd.to_datetime(df_covid_deaths_germany['Date'], format = '%Y-%m-%d')
# - Set date column as index 
df_covid_deaths_germany.set_index('Date', inplace=True)

# Step 2: Clean Data

# Step 3: Select only the necesarry columns from the dataframe 
df_covid_deaths_germany = df_covid_deaths_germany[['Number of Covid Deaths']]

# Step 4: Rename column names - not necessary

# Read File
df_covid_deaths_germany

Number of Covid Deaths
Date                              
2020-03-01                    1120
2020-04-01                    6069
2020-05-01                    1572
2020-06-01                     320
2020-07-01                     135
2020-08-01                     152
2020-09-01                     206
2020-10-01                    1480
2020-11-01                    8604
2020-12-01                   22035
2021-01-01                   21997
2021-02-01                    9803
2021-03-01                    5519
2021-04-01                    6572
2021-05-01                    4734
2021-06-01                    1089
2021-07-01                     276
2021-08-01                     491
2021-09-01                    1605
2021-10-01                    2491
2021-11-01                    8095
2021-12-01                   10396
2022-01-01                    4520
2022-02-01                    5664
2022-03-01                    7485
2022-04-01                    5232
2022-05-01                    2247
2022-06-01                    1511
2022-07-01                    3294
2022-08-01                    2747
2022-09-01                     760

In [6]:
# Pipeline Covid Intensive Care Cases

# File path
file_path = 'data/intensive care covid cases germany.csv'

# Open File
df_ic_cases_covid_germany = pd.read_csv(file_path,  delimiter=',')


# Adjust Data
# Step 1: Convert date data from datetime to date
for i in range(len(df_ic_cases_covid_germany)):
    d = df_ic_cases_covid_germany['date'][i] 
    df_ic_cases_covid_germany['date'][i] = datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z').date()


# Step 2: Clean Data

# Step 3: Select only the necesarry columns from the dataframe 

# Step 4: Rename column names
df_ic_cases_covid_germany = df_ic_cases_covid_germany.rename(columns={
    'date': 'Date',
    'COVID-19-Fälle': 'Covid Cases on IC'})

df_ic_cases_covid_germany.set_index('Date', inplace=True)
# Read File
df_ic_cases_covid_germany

# ------------ Remark ---------------
# I have no idea why this error is showing, but is the only way I can convert the datetime value to a date value
# ERROR: 
    # A value is trying to be set on a copy of a slice from a DataFrame
    #
    # See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
    # df_ic_cases_covid_germany['date'][i] = datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z').date()

C:\Users\larsk\AppData\Local\Temp\ipykernel_40884\3601102438.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ic_cases_covid_germany['date'][i] = datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z').date()


Covid Cases on IC
Date                         
2020-03-20                200
2020-03-21                308
2020-03-22                364
2020-03-23                451
2020-03-24                616
...                       ...
2022-10-05               1294
2022-10-06               1344
2022-10-07               1366
2022-10-08               1406
2022-10-09               1453

[934 rows x 1 columns]

# Data analysis 

## Road safety

In [12]:
# Step 1: select data
df1 = df_road_safety_germany

#Step 2: create line-plot
fig = px.line(df1, x= df_road_safety_germany.index, y= "Accidents involving human injury",
             title = "Figure 1: Weekly total accidents traffic accidents Germany")
fig.show()

In [8]:
# Step 1: select data
df2 = df_accidents_human_injury

# Step 2: Sum up all the different categories of incidents involving human injuries
total_withincitylimits = 0
for i in df_accidents_human_injury['Accidents involving human injury - within city limits']:
    total_withincitylimits = total_withincitylimits + i

total_highway = 0
for i in df_accidents_human_injury['Accidents involving human injury - highway']:
    total_highway = total_highway + i

total_outsidecitylimits = 0
for i in df_accidents_human_injury['Accidents involving human injury - outside city limits, off highway']:
    total_outsidecitylimits = total_outsidecitylimits + i

#Step 3: Create bar-plot
categories = ['total_withincitylimits', 'total_highway', 'total_outsidecitylimits']
accidents = [total_withincitylimits, total_highway, total_outsidecitylimits]

fig = px.bar(x = categories, y = accidents)
fig.show()

## COVID-data

In [9]:
# Step 1: Select data 
df2 = df_positive_covid_test_germany

# Step 2: Create line-plot
fig = px.line(df2, x= df_positive_covid_test_germany.index, 
              y= "Number of Positive COVID-19 tests per 100.000 capita",
              title = "Figure 2: Total positive COVID-tests Germany")
fig.update_traces(line_color='#EF553B')
fig.show()

In [10]:
# Step 1: Select data 
df3 = df_covid_deaths_germany

# Step 2: Create line-plot
fig = px.line(df3, x= df_covid_deaths_germany.index, y= "Number of Covid Deaths", 
              title = "Figure 3: Total COVID-Deaths Germany")
fig.update_traces(line_color='#EF553B')
fig.show()

In [11]:
# Step 1: Select data
df4 = df_ic_cases_covid_germany

# Step 2: Create line-plot
fig = px.line(df4, x= df_ic_cases_covid_germany.index, y= "Covid Cases on IC", 
              title="Figure 4: Total COVID-cases on IC" )
fig.update_traces(line_color='#EF553B')
fig.show()

In [105]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x = df2.index, 
           y = df2['Number of Positive COVID-19 tests per 100.000 capita'], 
           name = 'Positive tests previous 7 days per 100.000 capita', 
           marker_color = 'blue',
           opacity = 0.6),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x = df3.index, 
           y = df3['Number of Covid Deaths'], 
           name = 'Monthly Covid Deaths Germany', 
           marker_color = 'red',
           opacity = 0.6),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x = df4.index, 
            y = df4['Covid Cases on IC'], 
            name = 'Daily Covid IC cases Germany',
            line_color = 'black'),
    secondary_y=False,
)
# Add figure title
fig.update_layout(
    title_text="Complete Covid Analysis"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Covid Deaths & IC Cases</b>", secondary_y=False)
fig.update_yaxes(range=[0,25000], secondary_y=False)
fig.update_yaxes(title_text="<b>Covid Tests</b>", secondary_y=True)
fig.update_yaxes(range=[0,2000], secondary_y=True)

fig.show()

In [156]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x = df2.index, 
           y = df2['Number of Positive COVID-19 tests per 100.000 capita'], 
           name = 'Positive tests previous 7 days per 100.000 capita', 
           marker_color = 'blue',
           opacity = 0.6),
    secondary_y=True,
)

fig.add_trace(
    go.Bar(x = df3.index, 
           y = df3['Number of Covid Deaths'], 
           name = 'Monthly Covid Deaths Germany', 
           marker_color = 'red',
           opacity = 0.6),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x = df4.index, 
            y = df4['Covid Cases on IC'], 
            name = 'Daily Covid IC cases Germany',
            line_color = 'black'),
    secondary_y=False,
)

fig.add_trace(
    go.Line(x = df1.index, 
            y = df1['Accidents involving human injury'], 
            name = 'Accidents involving human injury',
            line_color = 'purple'),
    secondary_y=False,
)
# Add figure title
fig.update_layout(
    title_text="Complete Covid Analysis"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="<b>Covid Deaths & IC Cases</b>", secondary_y=False)
fig.update_yaxes(range=[0,35000], secondary_y=False)
fig.update_yaxes(title_text="<b>Covid Tests</b>", secondary_y=True)
fig.update_yaxes(range=[0,2000], secondary_y=True)

# Set x axes limits
#fig.update_xaxes(range=["2018-01-01","2018-12-31"])

#fig.show()
fig.show()

C:\Users\larsk\anaconda3\envs\TIL6022\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


